In [5]:
from utils import *
import os
import numpy as np
from griffin_lim import *
import matplotlib.pyplot as plt
import soundfile as sf
from tqdm import tqdm

# Reconstrunction of original signals from retrieved spectrograms through Griffin-Lim

## Audio MNIST

In [ ]:
# audio mnist signal reconstrunction using griffin lim

win_sizes = [512, 1024, 2048] # specify the window sizes
sr = 48000 # specify the sample rate

for win_size in win_sizes:

    # gather all the files in the folder
    folder_path = f"results_mat/iDLG_audio_mnist{str(win_size)}"
    all_files = os.listdir(folder_path) 

    # specify parameters for the stft and istft
    hop = win_size // 2
    win = 'hamming'
    i = 0

    # iterate over all the files in the folder
    for file in all_files:
        spec_db = np.load(f'{folder_path}/{file}')
        inverse_spec = np.array(10**(spec_db[0][0]/20))
        try:
            y_reconstructed = mfgla(inverse_spec, iterations=1000, stft_kwargs={'n_fft': win_size, 'hop_length': hop, 'window' : win, 'center' : None }
                                , istft_kwargs={'n_fft': win_size, 'hop_length': hop, 'window' : win, 'center' : None })
        except Exception as e:
            print(f"Error processing {file} - {e}")
            continue
        
        # save reconstructed audio as an .wav file
        sf.write(f'results_audio/audio_mnist{str(win_size)}/' + file.replace("npy","wav"), y_reconstructed, sr)
        
        print(f"Processed {i}/{len(all_files)} files")
        i += 1

## Urbansound8K

Load all files, and gather samplerate for each file in a dictionary.

In [6]:
audio_data = load_data("data/audio")
audio_data = [data for data in audio_data if len(data[0]) in [176400, 192000]]

In [ ]:
sample_rate_dict = {file_name: sample_rate for _, sample_rate, file_name in audio_data}
print(sample_rate_dict)

Griffin-Lim of all files.

In [ ]:
# urbansound signal reconstrunction using griffin lim

win_sizes = [512,1024, 2048]
for win_size in win_sizes:

    # gather all the files in the result folder
    folder_path = f"results_mat/iDLG_urbansound{str(win_size)}"
    all_files = os.listdir(folder_path)

    # specify parameters for the stft and istft
    hop = win_size // 2
    win = 'hamming'
    i = 0

    # iterate over all the files in the folder
    for file in all_files:
        spec_db = np.load(f'{folder_path}/{file}')
        inverse_spec = np.array(10**(spec_db[0][0]/20))
        try:
            y_reconstructed = mfgla(inverse_spec, iterations=1000, stft_kwargs={'n_fft': win_size, 'hop_length': hop, 'window' : win, 'center' : None }
                                , istft_kwargs={'n_fft': win_size, 'hop_length': hop, 'window' : win, 'center' : None })
        except Exception as e:
            print(f"Error processing {file} - {e}")
            continue

        # gater the sample rate of the file
        sr = sample_rate_dict[file.replace("npy","wav")]

        # save file as an .wav file
        sf.write(f'results_audio/urbansound{str(win_size)}/' + file.replace("npy","wav"), y_reconstructed, sr)
        
        print(f"Processed {i}/{len(all_files)} files")
        i += 1